<a href="https://colab.research.google.com/github/pedrodeparis/Machine-Learning---Python/blob/main/Case_Insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
seguro = pd.read_excel("insurance.xlsx")
seguro.head()

,idade,sexo,imc,quantidade_filhos,fumante,regiao,custos_seguro
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.560,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
seguro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1341 entries, 0 to 1340
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   idade              1341 non-null   int64  
 1   sexo               1338 non-null   object 
 2   imc                1341 non-null   float64
 3   quantidade_filhos  1341 non-null   int64  
 4   fumante            1341 non-null   object 
 5   regiao             1341 non-null   object 
 6   custos_seguro      1341 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.5+ KB


In [4]:
seguro.dropna(inplace=True)

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(seguro.sexo)
seguro.sexo = le.transform(seguro.sexo)

le.fit(seguro.fumante)
seguro.fumante = le.transform(seguro.fumante)


le.fit(seguro.regiao)
seguro.regiao = le.transform(seguro.regiao)

In [7]:
seguro = pd.DataFrame(seguro)

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

seguro_norm = pd.DataFrame(scaler.fit_transform(seguro), index=seguro.index, columns=seguro.columns)
seguro_norm.head()

,idade,sexo,imc,quantidade_filhos,fumante,regiao,custos_seguro
0,0.021739,0.0,0.321227,0.0,1.0,1.000000,0.251611
1,0.000000,1.0,0.479150,0.2,0.0,0.666667,0.009636
2,0.217391,1.0,0.473500,0.6,0.0,0.666667,0.053115
3,0.326087,1.0,0.181464,0.0,0.0,0.333333,0.333010
4,0.304348,1.0,0.347592,0.0,0.0,0.333333,0.043816


In [15]:
import statsmodels.formula.api as smf
function = "custos_seguro~idade+sexo+imc+quantidade_filhos+fumante+regiao"
model = smf.ols(formula=function, data=seguro_norm).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          custos_seguro   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.750
Method:                 Least Squares   F-statistic:                     668.1
Date:                Mon, 15 Jan 2024   Prob (F-statistic):               0.00
Time:                        18:45:42   Log-Likelihood:                 1230.4
No. Observations:                1338   AIC:                            -2447.
Df Residuals:                    1331   BIC:                            -2410.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.0479      0.00

In [16]:
function = "custos_seguro~idade+imc+quantidade_filhos+fumante+regiao"
model = smf.ols(formula=function, data=seguro_norm).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          custos_seguro   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.750
Method:                 Least Squares   F-statistic:                     802.2
Date:                Mon, 15 Jan 2024   Prob (F-statistic):               0.00
Time:                        18:45:51   Log-Likelihood:                 1230.3
No. Observations:                1338   AIC:                            -2449.
Df Residuals:                    1332   BIC:                            -2417.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.0488      0.00

In [17]:
X = seguro_norm[["idade", "imc", "quantidade_filhos", "fumante", "regiao"]]

y = seguro_norm[["custos_seguro"]]

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

lr = LinearRegression()

lr.fit(X_train, y_train)

LinearRegression()

In [19]:
from sklearn import metrics
r_sq = lr.score(X,y)
print(r_sq)

0.7505629738411864


In [20]:
y_pred_train = lr.predict(X_train)
print("MAE", metrics.mean_absolute_error(y_train, y_pred_train))

MAE 0.06760649849946623


In [21]:
y_pred_test = lr.predict(X_test)
print("MAE", metrics.mean_absolute_error(y_test, y_pred_test))

MAE 0.06275502946015028


In [22]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

<ipython-input-22-0e2b706809eb>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


RandomForestRegressor()

In [26]:
r_sq = rf.score(X,y)
print(r_sq)
y_pred_train = rf.predict(X_train)
print("MAE", metrics.mean_absolute_error(y_train, y_pred_train))

y_pred_test = rf.predict(X_test)
print("MAE", metrics.mean_absolute_error(y_test, y_pred_test))

0.951891831776683
MAE 0.016470626132808447
MAE 0.04378870739934245


In [28]:
from sklearn.ensemble import AdaBoostRegressor
ada = AdaBoostRegressor()
ada.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor()

In [29]:
ada_pred_train = ada.predict(X_train)


r_sq = ada.score(X,y)
print(r_sq)
y_pred_train = ada.predict(X_train)
print("MAE", metrics.mean_absolute_error(y_train, y_pred_train))

y_pred_test = ada.predict(X_test)
print("MAE", metrics.mean_absolute_error(y_test, y_pred_test))

0.8281723569231935
MAE 0.06514182099060425
MAE 0.06493533507568369


In [30]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor()

gbr.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor()

In [31]:
gbr_pred_train = gbr.predict(X_train)


r_sq = gbr.score(X,y)
print(r_sq)
y_pred_train = gbr.predict(X_train)
print("MAE", metrics.mean_absolute_error(y_train, y_pred_train))

y_pred_test = gbr.predict(X_test)
print("MAE", metrics.mean_absolute_error(y_test, y_pred_test))

0.8972761174487603
MAE 0.03346236453828395
MAE 0.03925860415467806


#GBR performou melhor. Vamos melhorar os parâmetros.

In [44]:
from sklearn.model_selection import GridSearchCV

parameters = {
              "max_depth": [5],
              "min_samples_leaf": [4],
              "min_samples_split": [2],
              "n_estimators": [200]}

grid_search = GridSearchCV(gbr, parameters, scoring="r2", cv=2, n_jobs=-1)

In [45]:
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=2, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'max_depth': [5], 'min_samples_leaf': [4],
                         'min_samples_split': [2], 'n_estimators': [200]},
             scoring='r2')

In [46]:
print(grid_search.best_estimator_)
print(grid_search.best_params_)

GradientBoostingRegressor(max_depth=5, min_samples_leaf=4, n_estimators=200)
{'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}


In [47]:
best_model = grid_search.best_estimator_

In [48]:
best_model.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 5,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [54]:
gbr_tunned = GradientBoostingRegressor(
 alpha = 0.09,
 ccp_alpha = 0.0,
 criterion = 'friedman_mse',
 init = None,
 learning_rate = 0.1,
 loss = 'squared_error',
 max_depth = 5,
 max_features = 1.0,
 max_leaf_nodes = None,
 min_impurity_decrease = 0.0,
 min_samples_leaf = 4,
 min_samples_split = 2,
 min_weight_fraction_leaf = 0.0,
 n_estimators = 200,
 n_iter_no_change = None,
 random_state = None,
 subsample = 1.0,
 tol = 0.0001,
 validation_fraction = 0.1,
 verbose = 0,
 warm_start = False)

In [57]:
gbr_tunned.fit(X_train, y_train)
rf_tunned_pred_train = gbr_tunned.predict(X_train)
rf_tunned_pred_test = gbr_tunned.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [65]:
r_sq = gbr_tunned.score(X,y)

print(r_sq)
print("MAE", metrics.mean_absolute_error(y_train, rf_tunned_pred_train))
print("MAE", metrics.mean_absolute_error(y_test, rf_tunned_pred_test))

0.9325726845597542
MAE 0.0238308660696551
MAE 0.04571317753511194
